In [4]:
import gradio as gr
import pandas as pd
import json 
from FactureModel import Facture
import constants as const
from gradio import Interface, Image, Checkbox, Textbox
from functions import readDataSet, imageObjCreation, readImgOCR, pydanticParser, instructionsFormat, LLMModelCall

def process_invoice(image_path, do_easyocr):
    # Step 1: Process the image to extract text using OCR
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    image_obj = imageObjCreation(image_bytes)
    ocr_text = readImgOCR(image_bytes, image_obj, do_easyocr)

    # Step 2: Setup Pydantic parser and format instructions
    parser = pydanticParser()
    system_instructions, prompt = instructionsFormat(parser, ocr_text)

    # Step 3: Call the LLM model to extract invoice data
    extracted_data = LLMModelCall(prompt, system_instructions)
    facture_data = parser.parse(extracted_data)
   
    date = facture_data.date
    monto = facture_data.monto
    facture_number = facture_data.facture_number

    return date,monto,facture_number

# Define the Gradio interface
iface = gr.Interface(
    fn=process_invoice,
    inputs=[
        gr.Image(type="filepath", label="Upload Invoice Image"),
        gr.Checkbox(label="Use EasyOCR"),
    ],
     outputs=[
        gr.Textbox(label="Date"),
        gr.Textbox(label="Monto"),
        gr.Textbox(label="Facture Number")],
    title="Invoice Data Extractor",
    description="Upload an invoice image and extract data from it using OCR and LLM",
)

# Launch the Gradio app
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [2]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


# Gradio with Query and OCR 

In [ ]:
# import gradio as gr
# import pandas as pd
# import json 
# from FactureModel import Facture
# import constants as const
# from gradio import Interface, Image, Checkbox, Textbox
# from functions import readDataSet, imageObjCreation, readImgOCR, pydanticParser, instructionsFormat, LLMModelCall, query_pinecone

# def process_invoice(image_path, do_easyocr):
#     # Step 1: Process the image to extract text using OCR
#     with open(image_path, "rb") as f:
#         image_bytes = f.read()
#     image_obj = imageObjCreation(image_bytes)
#     ocr_text = readImgOCR(image_bytes, image_obj, do_easyocr)

#     # Step 2: Setup Pydantic parser and format instructions
#     parser = pydanticParser()
#     system_instructions, prompt = instructionsFormat(parser, ocr_text)

#     # Step 3: Call the LLM model to extract invoice data
#     extracted_data = LLMModelCall(prompt, system_instructions)
#     facture_data = parser.parse(extracted_data)
   
#     date = facture_data.date
#     monto = facture_data.monto
#     facture_number = facture_data.facture_number

    
#     return date,monto,facture_number

# def query_database(query_vector_text):
#     try:
#         query_vector= json.loads(query_vector_text)
#         db_results = query_pinecone(query_vector)
#     except json.JSONDecodeError:
#         db_results = "Invalid query vector format. Please enter a valid JSON array."
#     except Exception as e:
#         db_results = str(e)

#     return db_results
    
# def main_menu(choice):
#     if choice == "Extract Invoice Data":
#         return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
#     elif choice == "Query Pinecone Database":
#         return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)
    
# menu_choices = ["Extract Invoice Data", "Query Pinecone Database"]
# with gr.Blocks() as demo:
#     with gr.Row():
#         dropdown = gr.Dropdown(menu_choices, label="Choose an option")
#         submit = gr.Button("Submit")

#     with gr.Row(visible=False) as invoice_interface_row:
#         with gr.Column():
#             invoice_interface = gr.Interface(
#                 fn=process_invoice,
#                 inputs=[
#                     gr.Image(type="filepath", label="Upload Invoice Image"),
#                     gr.Checkbox(label="Use EasyOCR"),
#                 ],
#                 outputs=[
#                     gr.Textbox(label="Date"),
#                     gr.Textbox(label="Monto"),
#                     gr.Textbox(label="Facture Number")
#                 ],
#                 title="Invoice Data Extractor",
#                 description="Upload an invoice image and extract data from it using OCR and LLM"
#             ).render()

#     with gr.Row(visible=False) as pinecone_interface_row:
#         with gr.Column():
#             pinecone_query_interface = gr.Interface(
#                 fn=query_database,
#                 inputs=[
#                     gr.Textbox(label="Query Vector", placeholder="Enter query vector as a list of numbers")
#                 ],
#                 outputs=[
#                     gr.Textbox(label="Pinecone Results")
#                 ],
#                 title="Pinecone Database Query",
#                 description="Enter a query vector to search the Pinecone database."
#             ).render()

#     submit.click(
#         main_menu,
#         inputs=[dropdown],
#         outputs=[dropdown, invoice_interface_row, pinecone_interface_row]
#     )

# # Launch the main menu
# demo.launch()